create initial metadata file

In [26]:
import glob, json
from wavinfo import WavInfoReader
from engine.files.files import File
from engine.helpers import write_file

encoding = 'utf-8'

def sanatize_bytes(bytes):
	# first_null = next( (index for index, byte in enumerate(bytes) if byte == 0 ), None )
	# if first_null is not None:
	# 	trimmed = bytes[:first_null]
	# else:
	# 	trimmed = bytes

	decoded = bytes.decode(encoding, errors='replace')
	return decoded

label_mapping = {'ps': 'UPS', 'T': 'UPS'}
def normalize_label(label):
    if label in label_mapping:
        return label_mapping[label]
    return label


jsonMeta = dict()

for path in glob.glob('../data/audio/simpleCall2/*'):
    if path.endswith('.wav'):
        info = WavInfoReader(path)
        with open(path, 'rb') as f:
            lblChunk = info._find_chunk_data(b'LBL ', f, True)
            if lblChunk == None:
                lblChunk = info._find_chunk_data(b'TIME', f)

            data_list = sanatize_bytes(lblChunk).split("\r\n")
            print(data_list)
            data_list = list(filter(lambda item: len(item)>1, [i.split("\t") for i in data_list]))

        labels = []
        for entry in data_list:
            # entry = data_list[index]
            entry = [value.strip().replace("\u0000", '') for value in entry]
            labels.append({
                'start': float(entry[0])
                , 'end': float(entry[1])
                , 'sequence': normalize_label(entry[2])
            })

        jsonEntry = {
            'audio': path[8:].replace('\\', '/') #remove ../data/
            , 'duration': info.data.frame_count / info.fmt.sample_rate
            , 'labels': labels
            , 'label': 'none'
        }

        jsonMeta[File(path).p.stem] = jsonEntry

write_file(f"../data/audio/simpleCall2/metadata.json", json.dumps(jsonMeta, indent=2))



['                                                               ', '1.684022e-002\x00\t6.335211e-002\x00\tB3                             \tM', '']
['                                                               ', '7.765001e-002\x00\t1.345640e-001\x00\tES                             \tM', '4.889860e-001\x00\t5.492360e-001\x00\tES                             \tM', '4']
['                                                               ', '6.475067e-002\x00\t7.804871e-002\x00\tVSV                            \tM', '']
['                                                               ', '4.093170e-002\x00\t5.523682e-002\x00\tVSV                            \tM', '9.563446e-002\x00\t1.104355e-001\x00\tVSV                            \tM', '1']
['                                                               ', '2.026367e-002\x00\t3.306580e-002\x00\tVSV                            \tM', '']
['                                                               ', '3.454400e-001\x00\t3.569600e-001\x00\

create split parts


In [8]:
import json
from engine.helpers import write_file
from engine.audio import load_audio
from engine.metadata import metadata_db

mdb = metadata_db('audio/simpleCall/metadata.json').db
min_duration = 1
for stem in mdb:
    entry = mdb[stem]
    for label in entry.labels:
        min_duration = min(min_duration, label['end']- label['start'])

print(min_duration)


0.005200150000000001


In [ ]:
load_audio(mdb, 'simple_call_2', 'audio/simpleCall').extract_label_parts('simple_call_2', True).create_spectrogram(sampling_rate=500000, x_pixels_per_sec=2000, height=512, window='Ham').run()


In [20]:
import re
from collections import defaultdict

task = load_audio(mdb, 'simple_call_2', 'audio/simpleCall2').run()
index = defaultdict(list)
p = re.compile('(([^_]*_){2}(\d{2}_){3})(\d*)')
for file in task.files:
    match = p.findall(file.p.stem)
    key = match[0][0]
    # key = file.p.stem.split(match[0][1])[0]
    index[key].append(file)

for key in index:
    files = index[key]
    duration = 0
    for file in files:
        duration += file.metadata.duration
    print(key,len(files), duration)


Loaded audio tracks from audio/simpleCall
Already processed: audio\simpleCall\B2_C_B_Jt1_16_07_06_042.wav, audio\simpleCall\B2_C_TR_Jt1_16_06_02_018.wav, audio\simpleCall\B2_C_TR_Jt2_16_06_02_044.wav, ... (299 others)
B_Jt1_16_07_06_ 3 2.412442
TR_Jt1_16_06_02_ 10 87.71842000000001
TR_Jt2_16_06_02_ 7 107.323002
TR_Jt5_16_07_10_ 1 2.060072
TO_Jt1_16_06_20_ 2 610.831744
TO_Jt2_16_06_12_ 5 234.57691
TO_Jt2_16_06_18_ 1 54.576832
TO_Jt2_16_06_20_ 3 94.49250799999999
TO_Jt2_16_06_24_ 5 75.769138
TO_Jt2_16_06_25_ 1 121.62016
TO_Jt2_16_06_26_ 10 396.75971599999997
TO_Jt2_16_07_04_ 3 256.237478
TO_Jt2_16_07_06_ 3 287.87870399999997
TO_Jt3_16_06_18_ 4 117.320584
TO_Jt3_16_06_20_ 1 38.71712
TO_Jt3_16_06_24_ 3 105.581376
TO_Jt3_16_07_04_ 3 182.228408
TO_Jt3_16_07_06_ 6 401.626958
TO_Jt3_16_07_09_ 1 59.950784
TO_Jt3_16_07_11_ 9 436.971304
TO_Jt3_16_07_13_ 1 302.8272
TR_Jt1_16_06_07_ 5 315.80234599999994
TR_Jt2_16_06_07_ 1 0.000806
TR_Jt3_16_06_16_ 1 188.892864
TR_Jt4_16_06_11_ 2 191.911696
TR_Jt4_1

In [3]:
# load_audio(mdb, 'simple_call_2', 'audio/simpleCall2').create_spectrogram(sampling_rate=500000, x_pixels_per_sec=2000, height=265, window='Ham').run()

list = load_audio(mdb, 'simple_call_2', 'audio/simpleCall2').create_silent_derivatives(.2).run()
for file in list.files:
    mdb.db[file.p.stem] = dict()

load_audio(mdb, 'simple_call_2', 'audio/simpleCall2/silent').create_spectrogram(sampling_rate=500000, x_pixels_per_sec=2000, height=265, window='Ham').run()

Create spectrograms: 100%|██████████| 4/4 [00:01<00:00,  3.52it/s]


Loaded audio tracks from audio/simpleCall2
Already processed: audio\simpleCall2\silent\B_Jt1_16_07_06_combined.wav, audio\simpleCall2\silent\B_Jt1_16_06_14_combined.wav, audio\simpleCall2\silent\TR_Jt5_16_06_21_combined.wav, ... (1 others)
Loaded audio tracks from audio/simpleCall2/silent
Already processed: audio\simpleCall2\silent\B_Jt1_16_06_14_combined.wav, audio\simpleCall2\silent\B_Jt1_16_07_06_combined.wav, audio\simpleCall2\silent\TR_Jt5_16_06_21_combined.wav, ... (1 others)
